In [9]:
from packages.database_tools import connect
conn, cur = connect()

import pandas as pd

Connected to PostgreSQL


In [ ]:
from collections import defaultdict

def fetch_job_results(job_id=None):
    if job_id is None:
        query = """
        SELECT * FROM games
        WHERE job_id = (SELECT job_id FROM games WHERE finished_on IS NOT NULL ORDER BY finished_on DESC LIMIT 1)
        """
    else:
        query = f"""
        SELECT * FROM games
        WHERE job_id = '{job_id}'
        """
    
    cur.execute(query)
    results = cur.fetchall()
    df = pd.DataFrame(results, columns=[desc[0] for desc in cur.description])
    return df

def get_deck_stats(df):
    combination_stats = defaultdict(lambda: {'games': 0, 'wins': 0})
    single_stats = defaultdict(lambda: {'games': 0, 'wins': 0})
    
    for _, row in df.iterrows():
        # Deck 1
        d1 = row['deck1_name']
        d1_wins = row['deck1_wins']
        if d1:
            combination_stats[d1]['games'] += row['game_count']
            combination_stats[d1]['wins'] += d1_wins
            # Sub-decks
            for part in d1.split():
                single_stats[part]['games'] += row['game_count']
                single_stats[part]['wins'] += d1_wins

        # Deck 2
        d2 = row['deck2_name']
        d2_wins = row['deck2_wins']
        if d2:
            combination_stats[d2]['games'] += row['game_count']
            combination_stats[d2]['wins'] += d2_wins
            for part in d2.split():
                single_stats[part]['games'] += row['game_count']
                single_stats[part]['wins'] += d2_wins

        # Deck 3
        d3 = row['deck3_name']
        d3_wins = row['deck3_wins']
        if d3:
            combination_stats[d3]['games'] += row['game_count']
            combination_stats[d3]['wins'] += d3_wins
            for part in d3.split():
                single_stats[part]['games'] += row['game_count']
                single_stats[part]['wins'] += d3_wins

        # Deck 4
        d4 = row['deck4_name']
        d4_wins = row['deck4_wins']
        if d4:
            combination_stats[d4]['games'] += row['game_count']
            combination_stats[d4]['wins'] += d4_wins
            for part in d4.split():
                single_stats[part]['games'] += row['game_count']
                single_stats[part]['wins'] += d4_wins

    # Convert to DataFrame
    combination_stats_df = pd.DataFrame([
        {'deck': k, 'games': v['games'], 'wins': v['wins'], 'win_rate': v['wins'] / v['games'] if v['games'] > 0 else 0}
        for k, v in combination_stats.items()
    ])
    single_stats_df = pd.DataFrame([
        {'deck': k, 'games': v['games'], 'wins': v['wins'], 'win_rate': v['wins'] / v['games'] if v['games'] > 0 else 0}
        for k, v in single_stats.items()
    ])
    combination_stats_df = combination_stats_df.sort_values(by='games', ascending=False).reset_index(drop=True)
    single_stats_df = single_stats_df.sort_values(by='games', ascending=False).reset_index(drop=True)
    return combination_stats_df, single_stats_df

def get_subdeck_stats(df):
    stats = defaultdict(lambda: {'games': 0, 'wins': 0})

    for _, row in df.iterrows():
        for i in range(1, 5):
            deck_name = row.get(f'deck{i}_name')
            deck_wins = row.get(f'deck{i}_wins')
            game_count = row.get('game_count')
            if deck_name and isinstance(deck_name, str):
                for part in deck_name.split():
                    stats[part]['games'] += game_count
                    stats[part]['wins'] += deck_wins

    stats_df = pd.DataFrame([
        {'subdeck': k, 'games': v['games'], 'wins': v['wins'], 'win_rate': v['wins'] / v['games'] if v['games'] > 0 else 0}
        for k, v in stats.items()
    ])
    stats_df = stats_df.sort_values(by='games', ascending=False).reset_index(drop=True)
    return stats_df

In [11]:
cur.execute("SELECT job_id FROM games WHERE finished_on IS NOT NULL ORDER BY finished_on;")
job_id = cur.fetchone()[0]
job_id

'f4a32d0d-7ed1-4502-a897-93f0cdc54eec'

In [13]:
df = fetch_job_results(job_id)

InFailedSqlTransaction: current transaction is aborted, commands ignored until end of transaction block


In [ ]:
combo_df, single_df = get_deck_stats(df)

In [ ]:
combo_df.head()

,deck,games,wins,win_rate
0,Aggressive Bloodthirsty,3250,746,0.229538
1,Beefy Aggressive,3250,26,0.008000
2,Bloodthirsty Dinos,3250,28,0.008615
3,Goblins Dinos,3250,0,0.000000
4,Necromancy Bloodthirsty,3250,0,0.000000


In [ ]:
single_df.head()

,deck,games,wins,win_rate
0,Aggressive,35750,883,0.024699
1,Bloodthirsty,35750,869,0.024308
2,Flourishing,35750,133,0.003720
3,Beefy,35750,293,0.008196
4,Gambling,35750,65,0.001818


In [ ]:
df_high_wins = df[(df['deck1_wins'] > 9) | (df['deck2_wins'] > 9)]
df_high_wins

,primary_key,deck1_name,deck2_name,deck3_name,deck4_name,job_id,game_count,deck1_wins,deck2_wins,deck3_wins,deck4_wins,turn_counts,device_id,format,created_on,finished_on
1,561c063f-f970-439f-96e0-afd827995d5b,Aggressive Bloodthirsty,Beefy Gambling,None,None,89fae921-6800-4f4c-9839-5647476403c6,50,24,26,0,0,"[8, 7, 13, 13, 10, 7, 9, 9, 9, 9, 11, 7, 8, 7,...",9f229602-c78c-4859-b4c6-b53fbc1c5042,jumpstart,2025-08-20 19:13:30.560035,2025-08-20 23:25:01.753547
12,5b896178-c538-4f23-9c0c-d969752fb909,Aggressive Bloodthirsty,Beefy Flourishing,None,None,89fae921-6800-4f4c-9839-5647476403c6,50,23,27,0,0,"[8, 8, 11, 10, 7, 8, 9, 9, 10, 13, 8, 11, 9, 1...",9f229602-c78c-4859-b4c6-b53fbc1c5042,jumpstart,2025-08-20 19:13:30.560035,2025-08-20 23:25:30.000275
13,73ce55e1-5eeb-401a-ad09-204274047689,Aggressive Bloodthirsty,Flourishing Dragons,None,None,89fae921-6800-4f4c-9839-5647476403c6,50,31,19,0,0,"[9, 7, 9, 9, 14, 9, 12, 10, 8, 8, 7, 10, 6, 10...",c7980169-fdca-4130-a51a-719b66849bf0,jumpstart,2025-08-20 19:13:30.560035,2025-08-20 23:34:38.504337
15,f344f004-3f00-40aa-8a54-cdf3231bc71a,Aggressive Bloodthirsty,Beefy Morbid,None,None,89fae921-6800-4f4c-9839-5647476403c6,50,26,24,0,0,"[9, 7, 9, 9, 11, 8, 9, 8, 7, 9, 19, 14, 10, 10...",c7980169-fdca-4130-a51a-719b66849bf0,jumpstart,2025-08-20 19:13:30.560035,2025-08-20 23:33:59.266399
21,b514ed99-3dfe-4349-9ee4-50a77b6ae2d6,Aggressive Bloodthirsty,Flourishing Morbid,None,None,89fae921-6800-4f4c-9839-5647476403c6,50,30,20,0,0,"[9, 8, 10, 11, 10, 8, 10, 9, 7, 9, 10, 15, 6, ...",9f229602-c78c-4859-b4c6-b53fbc1c5042,jumpstart,2025-08-20 19:13:30.560035,2025-08-20 23:34:24.371204
30,77b66d95-5ab9-496a-8ac2-8bed5e741d18,Aggressive Bloodthirsty,Gambling Bloodthirsty,None,None,89fae921-6800-4f4c-9839-5647476403c6,50,30,20,0,0,"[8, 12, 12, 7, 10, 25, 8, 9, 11, 9, 8, 8, 8, 6...",9f229602-c78c-4859-b4c6-b53fbc1c5042,jumpstart,2025-08-20 19:13:30.560035,2025-08-20 23:33:04.206663
31,93c9eadf-6f0f-4415-be6c-d3731e310f1b,Aggressive Bloodthirsty,Dinos Dragons,None,None,89fae921-6800-4f4c-9839-5647476403c6,50,30,20,0,0,"[11, 9, 10, 7, 8, 8, 8, 7, 12, 7, 9, 8, 6, 7, ...",c7980169-fdca-4130-a51a-719b66849bf0,jumpstart,2025-08-20 19:13:30.560035,2025-08-20 23:31:43.952025
32,d63d05ca-a129-4b43-b415-f93750c6d353,Aggressive Bloodthirsty,Dinos Morbid,None,None,89fae921-6800-4f4c-9839-5647476403c6,50,29,21,0,0,"[11, 10, 9, 10, 7, 9, 6, 7, 8, 9, 8, 11, 10, 6...",c7980169-fdca-4130-a51a-719b66849bf0,jumpstart,2025-08-20 19:13:30.560035,2025-08-20 23:32:46.852698
34,51b7db88-a0a8-42df-aa21-f63711b1a02a,Aggressive Bloodthirsty,Bloodthirsty Dinos,None,None,89fae921-6800-4f4c-9839-5647476403c6,50,22,28,0,0,"[9, 8, 7, 10, 11, 8, 14, 8, 9, 11, 14, 8, 8, 9...",c7980169-fdca-4130-a51a-719b66849bf0,jumpstart,2025-08-20 19:13:30.560035,2025-08-20 23:33:23.876702
2120,d50c0f1e-c33f-402f-966c-b78b39205e50,Aggressive Bloodthirsty,Aggressive Dinos,None,None,89fae921-6800-4f4c-9839-5647476403c6,50,27,23,0,0,"[9, 10, 10, 10, 8, 7, 7, 6, 10, 9, 10, 8, 10, ...",9f229602-c78c-4859-b4c6-b53fbc1c5042,jumpstart,2025-08-20 19:13:30.560035,2025-08-20 23:17:57.665725
